In [ ]:
from transformers import AutoTokenizer
import coremltools as ct
import os
import numpy as np


model_path = "coreml_3B_INT4.mlpackage"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
        "meta-llama/Llama-3.2-3B-Instruct", token=""
    )

mlmodel_fp16 = ct.models.MLModel(model_path)

In [ ]:
inputs = tokenizer("hello world!", return_tensors='np')

tok = inputs['input_ids']

In [ ]:
st_len = tok.shape[-1]

state = mlmodel_fp16.make_state()  # 루프 내에서 상태 초기화

for i in range(10):

    mask = np.full((1, st_len:=st_len+1), -1e9)
    mask = np.triu(mask, k=1)
    mask = np.hstack(
        [np.zeros((1, 1)), mask]
    )[None, None, :, :]

    
    input_dict = {
        'input_ids': tok.astype(np.int32),
        'causal_mask': mask.astype(np.int32)

    }

    preds = mlmodel_fp16.predict(input_dict, state=state)
    
    logits = preds['logits']
    pre_toks = np.argmax(logits[0], axis=-1)

    tok = np.concatenate([tok, [[pre_toks]]], axis= 1)

print(tok)